In [9]:
#capture()

In [10]:
monitor(maxloops=2,delay=10)

after running for 0:00:23:
5 spectra
13 thermographs
1630 temperature datapoints

after running for 0:00:33:
8 spectra
23 thermographs
3179 temperature datapoints



In [3]:
#stop()
#temp_plot()
#power_plot()
#spectra_plot()

In [4]:
%matplotlib inline

#########################
### IMPORT STATEMENTS ###
#########################

from matplotlib import pyplot as plt
import sh
import os
import datetime
import numpy
import time

# synchronize time on all machines with:
# sudo service ntp stop && sudo ntpdate time.nrc.ca && sudo service ntp start

#### create the folders that will contain the data
droot = '/home/brandon/repos/bcduino/' # absolute paths
dbash = droot+'bash/'
ddata = droot+'data/'
dpython = droot+'python/'
started = datetime.datetime.now()
tstart = started.strftime("%Y-%m-%d_%H:%M:%S")
dexp = ddata+tstart + '/'
dspectra = dexp+'spectra/'
dtemp = dexp+'temperature/'
dthermo = dexp+'thermograph/'
dosc = dexp+'oscilloscope/'
dplot = dexp+'plots/'
sh.mkdir(dexp)
sh.mkdir(dspectra)
sh.mkdir(dtemp)
sh.mkdir(dthermo)
sh.mkdir(dosc)
sh.mkdir(dplot)
print(tstart)
print(os.listdir(dexp))

2016-05-03_07:26:50
['thermograph', 'plots', 'oscilloscope', 'temperature', 'spectra']


In [5]:
########################
### CAPTURE THE DATA ###
########################

def capture():
    
    ### run thermometry - WORKS
    stty = sh.Command('/bin/stty')
    cat = sh.Command('/bin/cat')
    stty('-F','/dev/ttyACM0','57600','raw')
    ftemp = dtemp+'temperature_'+tstart
    p = cat('/dev/ttyACM0',_bg=True,_out=ftemp)

    #### run thermography - WORKS
    cp = sh.Command("/bin/cp")
    cp(dbash+'thermograph', dthermo)
    capturethermograph = sh.Command(dthermo + 'thermograph')
    p = capturethermograph(_bg=True,_cwd=dthermo)

    #### run spectroscopy - WORKS
    # works great, but I don't know how to handle bash looping at the moment!
    #doceanoptics = droot+"c/oceanoptics/seabreeze-3.0.11/SeaBreeze/"
    #capturespectrum = sh.Command(doceanoptics+'sample-code/c/seabreeze-util')
    #p = capturespectrum("--get-spectrum","--integration-time-us","1000000"
    #                    ,_env={"LD_LIBRARY_PATH": doceanoptics+"lib"}
    #                    ,_bg=True
    #                    ,_out=dspectra+"spectrum_"+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))
    cp(dbash+'spectra.sh', dspectra)
    capturespectra = sh.Command(dspectra + 'spectra.sh')
    p = capturespectra(dspectra,_bg=True)

    #### run oscilloscope
    # this is not happening on the Pi3 yet, so don't worry about it
    # need to compile another kernel module to make this happen

In [6]:
################################
### MONITOR REALTIME LOGGING ###
################################

def monitor(maxloops, delay):
    """here is a function description"""
    loops = 0
    while (loops < maxloops):

        runtime = (datetime.datetime.now()-started) #.strftime("%Y-%m-%d_%H:%M:%S")
        print("after running for " + str(runtime).split('.')[0] + ":")

        ### check to make sure logging is occurring
        if (len(os.listdir(dspectra)) < 3):
            print("WARNING: spectroscopy down")
        else:
            print(str(len(os.listdir(dspectra))) + " spectra")

        if (len(os.listdir(dthermo)) < 3):
            print("WARNING: thermography down")
        else:
            print(str(len(os.listdir(dthermo))) + " thermographs")

        # not happening on the Pi3 yet
        #if (len(os.listdir(dosc)) < 3):             
        #    print("WARNING: oscilloscope down") 
        #else:
        #    print(len(os.listdir(dosc)) + " oscilloscope traces so far!")

        if (len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0])) < 10):
            print("WARNING: thermometry down")
        else:
            print(str(len(numpy.genfromtxt(dtemp+os.listdir(dtemp)[0]))) + " temperature datapoints")

        print('')
        time.sleep(delay) # give it a few seconds to log some more stuff
        loops = loops + 1


In [7]:
#######################
### STOP EVERYTHING ###
#######################
def stop():
    stop = sh.Command(dbash+'stoprun.sh')
    stop()

In [8]:
#######################################
### ANALYSIS AND PLOTTING FUNCTIONS ###
#######################################

###################################
### PLOTTING OSCILLOSCOPE STUFF ###
###################################

def getpower(vapp,vcap,title):
    windowSize = 1
    Liss = create_lissajous(Vapp=vapp,Vcap=vcap,window=windowSize)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.scatter([p[0] for p in Liss], [p[1] for p in Liss])
    area = find_area(Liss[0:1000])
    plt.title('The area of the Lissajous figure is: ' + str(area))
    ax.set_xlabel('volts, applied')
    ax.set_ylabel('volts, capacitor')
    plt.savefig(dplot+title + '_pwr' + str(area).split('.')[0] + '.png',dpi=150)
    plt.close(fig)
    
    volts = int(title)
    watts = int(str(area).split('.')[0])*0.47/1000.
    powers.append((volts,watts))

def moving_average(interval, window_size):
    """Finds the moving average of a dataset over a window size."""
    # algorithm via http://stackoverflow.com/questions/11352047/finding-moving-average-from-data-points-in-python
    window = numpy.ones(int(window_size))/float(window_size)
    return numpy.convolve(interval, window, 'same')

def data_import(fileName,fileSeparator,firstRow=1,xCol=1,yCol=2):
    """Imports data from .csv and puts it into an array."""
    dataFile = open(fileName, 'r')
    for i in range(0,firstRow):
        line = dataFile.readline()
    line = dataFile.readline()
    dataReturn = []
    while(line != ''): 
	lineSplit = line.split(fileSeparator)
        dataLine = tuple(float(item.strip()) for item in [lineSplit[int(xCol)-1],lineSplit[int(yCol)-1]])
        dataReturn.append(dataLine)
        line = dataFile.readline()
    return dataReturn
    
def data_pasted(dataString,fileSeparator,firstRow=1,xCol=1,yCol=2):
    """Prepares data that has been copied and pasted into the worksheet from a CSV file."""
    dataList = dataString.splitlines()
    dataReturn = []
    for line in dataList:
	lineSplit = line.split(fileSeparator)
        dataLine = tuple(float(item.strip()) for item in [lineSplit[int(xCol)-1],lineSplit[int(yCol)-1]])
        if len(dataLine)!=0: dataReturn.append(dataLine)
    return dataReturn

def find_area(array):
    """Find the array of a polygon defined as a set of Cartesian points in an array."""
    # algorithm via http://www.arachnoid.com/area_irregular_polygon/index.html
    a = 0
    ox,oy = array[0]
    for x,y in array[1:]:
        a += (x*oy-y*ox)
        ox,oy = x,y
    return abs(a/2)

def create_lissajous(Vapp,Vcap,window=1):
    """Generates a Lissajous figure from the applied and capacitor voltage oscilloscope traces."""
    #x=[Vapp[i][1] for i in range(0,len(Vapp))] # if your data also includes time
    #y=[Vcap[i][1] for i in range(0,len(Vcap))]
    x = Vapp
    y = Vcap
    if window!= 1:
        x = moving_average(x,window)
        y = moving_average(y,window)
    Liss = []
    for i in range(0,len(Vapp)):
        Liss.append((x[i],y[i]))
    return Liss

def power_extract():
    """determine the powers from the oscillscope data
    and write the (power,voltage) pairs to power.csv"""
    files = [item for item in sorted(os.listdir(dosc)) if '.png' not in item]
    powers = []
    for file in files:
        try:
            filename = dosc + file
            dataset = numpy.genfromtxt(filename,delimiter=',')
            #fig = plt.figure()
            #ax1 = fig.add_subplot(111)

            # http://matplotlib.org/examples/color/named_colors.html
            colors = ['yellow','cyan','pink','darkblue']
            labels = ['V','Vcap','I']

            xax = [line[0] for line in dataset]
            data_v = [line[1] for line in dataset]
            data_vcap = [line[2] for line in dataset]
            data_i = [line[3] for line in dataset]
            voltage = max(data_v)-min(data_v)

            getpower(vapp=data_v,vcap=data_vcap,title=str(voltage).split('.')[0])
        except:
            print("ERROR: " + filename)
    numpy.savetxt(dexp+'power.csv',powers,delimiter=',')

def power_plot():
    """plotting the data previously written to power.csv"""
    
    power_extract()

    data1 = numpy.genfromtxt(dexp+'power.csv',delimiter=',')
    #data2 = numpy.genfromtxt('power-pid-metal-1000sccm.csv',delimiter=',')

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot([item[0] for item in data1],[item[1] for item in data1],
           linestyle='None',marker='o',markerfacecolor='blue')
    #ax.plot([item[0] for item in data1],[item[1] for item in data1],
    #       linestyle='None',marker='o',markerfacecolor='blue',label='PID')
    #ax.plot([item[0] for item in data2],[item[1] for item in data2],
    #       linestyle='None',marker='o',markerfacecolor='red',label='Step')
    ax.set_xlabel('applied potential, V')
    ax.set_ylabel('plasma power, W')
    ax.set_title('plasma power vs applied potential')
    #ax.set_title('substrate: metal; gas: He 1000sccm; control: PID/tube temperature/voltage')
    #fig.show()
    fig.savefig(dplot+'power.png',dpi=150)
    plt.close(fig)

##################################
### PLOTTING TEMPERATURE STUFF ###
##################################

def temp_plot():
    files = sorted(os.listdir(dtemp)) #os.listdir('data'))
    for f in files:
        dataset = numpy.genfromtxt(dtemp+f,delimiter=',',skip_header=10,skip_footer=10,dtype=None)
        fig = plt.figure()
        ax = plt.subplot(111)

        # http://matplotlib.org/examples/color/named_colors.html
        colors = ['red','blue']
        labels = ['Temp1','Temp2']

        # convert the timestamps to runtime durations
        starttime = datetime.datetime.strptime(dataset[0][0],"%Y-%m-%d_%H:%M:%S")
        xax = [(datetime.datetime.strptime(line[0],"%Y-%m-%d_%H:%M:%S")-starttime).seconds for line in dataset]
        #print(xax[:5])
        #print(len(dataset[0]))

        for i in [1,2]:
        #for i in range(1,len(dataset[0])-1):
            data = [line[i] for line in dataset]
            if i == 0:
                voltage = max(data)-min(data)
            ax.plot(xax[:],data[:],
              linestyle='None',marker='o',markeredgecolor=colors[i-1],color=colors[i-1],markersize=4,
              label=labels[i-1])
        plt.legend()
        plt.ylim(0,95)
        plt.title("thermal traces")
        ax.set_xlabel('runtime, seconds')
        ax.set_ylabel('temperature, degC')
        fig.savefig(dplot+f+'.png',dpi=150)
        #fig.show()
        plt.close(fig)

###############################
### PLOTTING SPECTRAL STUFF ###
###############################

def spectra_plot():
    files = [item for item in sorted(os.listdir(dspectra)) if '.sh' not in item]
    for f in files:
        dataset = numpy.genfromtxt(dspectra+f,delimiter=',')
        fig = plt.figure()
        ax = plt.subplot(111)

        # http://matplotlib.org/examples/color/named_colors.html
        colors = ['red','blue']
        labels = ['']

        xax = [line[0] for line in dataset]
        #print(xax[:5])
        #print(len(dataset[0]))

        for i in [1,1]:
        #for i in range(1,len(dataset[0])-1):
            data = [line[i] for line in dataset]
            if i == 0:
                voltage = max(data)-min(data)
            ax.plot(xax[:],data[:],
              linestyle='None',marker='o',markeredgecolor=colors[i-1],color=colors[i-1],markersize=2,
              label=labels[i-1])
        plt.legend()
        plt.ylim(0,2500)
        plt.title("spectral traces")
        ax.set_xlabel('wavelength, nm')
        ax.set_ylabel('intensity, arb')
        fig.savefig(dplot+f+'.png',dpi=150)
        #fig.show()
        plt.close(fig)

###################################
### PLOTTING THERMOGRAPHY STUFF ###
###################################

# import previous efforts!